# Ruby OpenAI Agent Parallel Processing Example

This notebook demonstrates how to create multiple OpenAI agents and run them in parallel using the `async` gem.

## 1. Define the Agent Class

First, let's define a simple Agent class that will handle interactions with the OpenAI API.

In [1]:
class Agent
  attr_reader :name, :instructions, :model

  def initialize(name:, instructions:, model: "gpt-4o")
    @name = name
    @instructions = instructions
    @model = model
  end

  def run(message, client)
    prompt = [
      { role: "system", content: instructions },
      { role: "user", content: message }
    ]

    response = client.chat(
      parameters: {
        model: model,
        messages: prompt
      }
    )

    response.dig("choices", 0, "message", "content")
  end
end

:run

## 2. Create Different Agent Personalities

Now, let's create three different agents with unique personalities.

In [2]:
require "openai"
require "async"

# Initialize the OpenAI client with the API key from environment variables
client = OpenAI::Client.new(access_token: ENV["OPENAI_API_KEY"])

# Create three different agent personalities
agent1 = Agent.new(
  name: "Professional Sales Agent",
  instructions: "You are a very professional B2B salesperson who avoids jokes.",
  model: "gpt-4o-mini"
)

agent2 = Agent.new(
  name: "Engaging Sales Agent",
  instructions: "You are a playful, friendly salesperson who uses emoji and casual tone.",
  model: "gpt-4o-mini"
)

agent3 = Agent.new(
  name: "Busy Sales Agent",
  instructions: "You respond with short, curt responses because you are extremely busy.",
  model: "gpt-4o-mini"
)

agents = [agent1, agent2, agent3]

agents.each do |agent|
  puts "Created agent: #{agent.name} using model #{agent.model}"
end

Created agent: Professional Sales Agent using model gpt-4o-mini
Created agent: Engaging Sales Agent using model gpt-4o-mini
Created agent: Busy Sales Agent using model gpt-4o-mini


[#<#<Class:0x00007fb278326ea0>::Agent:0x00007fb278a48a30 @name="Professional Sales Agent", @instructions="You are a very professional B2B salesperson who avoids jokes.", @model="gpt-4o-mini">, #<#<Class:0x00007fb278326ea0>::Agent:0x00007fb278a48800 @name="Engaging Sales Agent", @instructions="You are a playful, friendly salesperson who uses emoji and casual tone.", @model="gpt-4o-mini">, #<#<Class:0x00007fb278326ea0>::Agent:0x00007fb278a485d0 @name="Busy Sales Agent", @instructions="You respond with short, curt responses because you are extremely busy.", @model="gpt-4o-mini">]

## 3. Run Agents in Parallel

Now, let's run all three agents in parallel using the `async` gem to generate sales emails.

In [3]:
message = "Write a cold sales email introducing our new AI tool."

results = []

# run all agents in parallel
Async do |task|
  tasks = agents.map do |agent|
    task.async { [agent.name, agent.run(message, client)] }
  end

  results = tasks.map(&:wait)
end

# display the results for testing
results.each do |name, output|
  puts "=== #{name} ===\n#{output}\n\n"
end

=== Professional Sales Agent ===
Subject: Transform Your Operations with Our New AI Tool

Dear [Recipient's Name],

I hope this message finds you well.

I am reaching out to introduce our latest AI tool, designed specifically to enhance operational efficiency and support data-driven decision-making for businesses like yours.

Our solution leverages advanced algorithms to streamline workflows, reduce manual processes, and improve accuracy in data analysis. Here are some key features:

- **Automation of Routine Tasks**: Save time and resources by automating repetitive tasks, allowing your team to focus on strategic initiatives.
- **Data Insights**: Gain actionable insights from your data in real-time, enabling more informed decision-making.
- **Scalability**: Our tool is designed to grow with your business, adapting to increasing demands without compromising performance.

We understand that integrating new technology can be challenging, which is why we offer comprehensive support during 

[["Professional Sales Agent", "Subject: Transform Your Operations with Our New AI Tool\n\nDear [Recipient's Name],\n\nI hope this message finds you well.\n\nI am reaching out to introduce our latest AI tool, designed specifically to enhance operational efficiency and support data-driven decision-making for businesses like yours.\n\nOur solution leverages advanced algorithms to streamline workflows, reduce manual processes, and improve accuracy in data analysis. Here are some key features:\n\n- **Automation of Routine Tasks**: Save time and resources by automating repetitive tasks, allowing your team to focus on strategic initiatives.\n- **Data Insights**: Gain actionable insights from your data in real-time, enabling more informed decision-making.\n- **Scalability**: Our tool is designed to grow with your business, adapting to increasing demands without compromising performance.\n\nWe understand that integrating new technology can be challenging, which is why we offer comprehensive sup

## 4. Compare and Analyze Results

Let's use another agent to analyze and compare the results from our three agents.

In [4]:
# extract all the email responses and format them for analysis
emails = results.map { |name, content| "#{name}:\n\n#{content}" }.join("\n\n---\n\n")

# create a prompt to analyze the emails
analysis_prompt = <<~PROMPT
  Here are three cold sales emails written by different AI agents:

  #{emails}

  Please analyze these emails and provide:
  1. A comparison of their styles and approaches
  2. The strengths and weaknesses of each
  3. Which one would likely be most effective and why
PROMPT

# Create an analyst agent
analyst = Agent.new(
  name: "Email Analyst",
  instructions: "You are an expert in sales and marketing communication who provides detailed, objective analysis.",
  model: "gpt-4o-mini"
)

# Run the analysis
analysis = analyst.run(analysis_prompt, client)

puts "=== Email Analysis ===\n#{analysis}"

=== Email Analysis ===
Certainly! Let's analyze the three cold sales emails based on their style, strengths, weaknesses, and potential effectiveness.

### 1. Comparison of Styles and Approaches

**Professional Sales Agent:**
- **Style:** Formal and structured. The tone is polite and professional, aimed at establishing credibility.
- **Approach:** Focuses on the features and benefits of the product in a logical manner, using bullet points for easy scanning. Emphasizes understanding of the recipient's challenges and offers support for onboarding.

**Engaging Sales Agent:**
- **Style:** Casual and conversational. Uses emojis and informal language to create a friendly tone.
- **Approach:** Positions the product as a fun and enriching addition to the recipient's workflow. Emphasizes the ease of use and the positive impact on productivity while encouraging further engagement through a personal touch.

**Busy Sales Agent:**
- **Style:** Concise and straightforward. It conveys information quic

## 5. Advanced: Create a Custom Agent Factory

Let's create a factory method to generate different types of agents more easily.

In [5]:
def create_agent(type, model: "gpt-4o-mini")
  case type
  when :professional
    Agent.new(
      name: "Professional Agent",
      instructions: "You are a very professional and formal business communicator.",
      model: model
    )
  when :creative
    Agent.new(
      name: "Creative Agent",
      instructions: "You are a highly creative and imaginative communicator who thinks outside the box.",
      model: model
    )
  when :technical
    Agent.new(
      name: "Technical Agent",
      instructions: "You are a technical expert who focuses on precise details and specifications.",
      model: model
    )
  when :empathetic
    Agent.new(
      name: "Empathetic Agent",
      instructions: "You are an empathetic communicator who focuses on emotional connection and understanding.",
      model: model
    )
  else
    raise "Unknown agent type: #{type}"
  end
end

new_agents = [:professional, :creative, :technical, :empathetic].map do |type|
  create_agent(type)
end

new_agents.each do |agent|
  puts "Created agent: #{agent.name}"
end

Created agent: Professional Agent
Created agent: Creative Agent
Created agent: Technical Agent
Created agent: Empathetic Agent


[#<#<Class:0x00007fb278326ea0>::Agent:0x00007fb276e93830 @name="Professional Agent", @instructions="You are a very professional and formal business communicator.", @model="gpt-4o-mini">, #<#<Class:0x00007fb278326ea0>::Agent:0x00007fb276e93678 @name="Creative Agent", @instructions="You are a highly creative and imaginative communicator who thinks outside the box.", @model="gpt-4o-mini">, #<#<Class:0x00007fb278326ea0>::Agent:0x00007fb276e93308 @name="Technical Agent", @instructions="You are a technical expert who focuses on precise details and specifications.", @model="gpt-4o-mini">, #<#<Class:0x00007fb278326ea0>::Agent:0x00007fb276e93178 @name="Empathetic Agent", @instructions="You are an empathetic communicator who focuses on emotional connection and understanding.", @model="gpt-4o-mini">]